![Pytorch](../../../pytorch_logo_2018.svg)

# Pytorch 中级篇（5）：语言模型（Language Model (RNN-LM)）

>参考代码
>
>**yunjey的 [pytorch tutorial系列](https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/main.py)**

## 语言模型 学习资料

语言模型这一块不是很想接触。就照着yunjey的代码，一带而过吧。

>**博客**
>
>[CS224d笔记4——语言模型和循环神经网络（Recurrent Neural Network, RNN）](https://wugh.github.io/posts/2016/03/cs224d-notes4-recurrent-neural-networks/?utm_source=tuicool&utm_medium=referral)
>
>[浅谈神经网络中的梯度爆炸问题](https://www.jianshu.com/p/79574b0f2959)

## Pytorch 实现

In [3]:
# 包
import torch
import torch.nn as nn
import numpy as np
from torch.nn.utils import clip_grad_norm
from data_utils import Dictionary, Corpus 

#data_utils代码在https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/language_model/data_utils.py

In [4]:
# 设备配置
# Device configuration
torch.cuda.set_device(1) # 这句用来设置pytorch在哪块GPU上运行
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# 超参数设置
# Hyper-parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000     # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

### Penn Treebank 数据集

In [6]:
corpus = Corpus()
ids = corpus.get_data('data/train.txt', batch_size)
vocab_size = len(corpus.dictionary)
num_batches = ids.size(1) // seq_length

### 基于RNN的语言模型

In [7]:
class RNNLM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(RNNLM, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, h):
        # Embed word ids to vectors
        x = self.embed(x)
        
        # Forward propagate LSTM
        out, (h, c) = self.lstm(x, h)
        
        # Reshape output to (batch_size*sequence_length, hidden_size)
        out = out.reshape(out.size(0)*out.size(1), out.size(2))
        
        # Decode hidden states of all time steps
        out = self.linear(out)
        return out, (h, c)

In [8]:
# 实例化一个模型
model = RNNLM(vocab_size, embed_size, hidden_size, num_layers).to(device)

In [9]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# 定义函数：截断反向传播
def detach(states):
    return [state.detach() for state in states] 

### 训练模型

In [11]:
for epoch in range(num_epochs):
    # 初始化隐状态和细胞状态
    states = ( torch.zeros(num_layers, batch_size, hidden_size).to(device),
                    torch.zeros(num_layers, batch_size, hidden_size).to(device) )
    
    for i in range(0, ids.size(1) - seq_length, seq_length):
        # Get mini-batch inputs and targets
        inputs = ids[:, i:i+seq_length].to(device)
        targets = ids[:, (i+1):(i+1)+seq_length].to(device)
        
        # Forward pass
        states = detach(states)
        outputs, states = model(inputs, states)
        loss = criterion(outputs, targets.reshape(-1))
        
        # Backward and optimize
        model.zero_grad()
        loss.backward()
        clip_grad_norm(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // seq_length
        if step % 100 == 0:
            print ('Epoch [{}/{}], Step[{}/{}], Loss: {:.4f}, Perplexity: {:5.2f}'
                   .format(epoch+1, num_epochs, step, num_batches, loss.item(), np.exp(loss.item())))

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Epoch [1/5], Step[0/1549], Loss: 9.2070, Perplexity: 9966.60
Epoch [1/5], Step[100/1549], Loss: 5.9989, Perplexity: 402.99
Epoch [1/5], Step[200/1549], Loss: 5.9188, Perplexity: 371.96
Epoch [1/5], Step[300/1549], Loss: 5.7725, Perplexity: 321.35
Epoch [1/5], Step[400/1549], Loss: 5.6823, Perplexity: 293.63
Epoch [1/5], Step[500/1549], Loss: 5.1482, Perplexity: 172.12
Epoch [1/5], Step[600/1549], Loss: 5.1709, Perplexity: 176.07
Epoch [1/5], Step[700/1549], Loss: 5.3420, Perplexity: 208.93
Epoch [1/5], Step[800/1549], Loss: 5.1762, Perplexity: 177.00
Epoch [1/5], Step[900/1549], Loss: 5.0525, Perplexity: 156.42
Epoch [1/5], Step[1000/1549], Loss: 5.0810, Perplexity: 160.94
Epoch [1/5], Step[1100/1549], Loss: 5.3304, Perplexity: 206.52
Epoch [1/5], Step[1200/1549], Loss: 5.1753, Perplexity: 176.85
Epoch [1/5], Step[1300/1549], Loss: 5.1375, Perplexity: 170.29
Epoch [1/5], Step[1400/1549], Loss: 4.8377, Perplexity: 126.18
Epoch [1/5], Step[1500/1549], Loss: 5.1570, Perplexity: 173.65
Epo

### 测试和保存模型

In [12]:
with torch.no_grad():
    with open('sample.txt', 'w') as f:
        # 初始化隐状态和细胞状态
        state = (torch.zeros(num_layers, 1, hidden_size).to(device),
                         torch.zeros(num_layers, 1, hidden_size).to(device))

        # 随机选择一个单词
        prob = torch.ones(vocab_size)
        input = torch.multinomial(prob, num_samples=1).unsqueeze(1).to(device)

        for i in range(num_samples):
            # Forward propagate RNN 
            output, state = model(input, state)

            # Sample a word id
            prob = output.exp()
            word_id = torch.multinomial(prob, num_samples=1).item()

            # Fill input with sampled word id for the next time step
            input.fill_(word_id)

            # File write
            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and save to {}'.format(i+1, num_samples, 'sample.txt'))

Sampled [100/1000] words and save to sample.txt
Sampled [200/1000] words and save to sample.txt
Sampled [300/1000] words and save to sample.txt
Sampled [400/1000] words and save to sample.txt
Sampled [500/1000] words and save to sample.txt
Sampled [600/1000] words and save to sample.txt
Sampled [700/1000] words and save to sample.txt
Sampled [800/1000] words and save to sample.txt
Sampled [900/1000] words and save to sample.txt
Sampled [1000/1000] words and save to sample.txt


In [13]:
# 保存模型
torch.save(model.state_dict(), 'model.ckpt')

**对于整个流程一脸懵逼，结果也不是很懂**